# Read files

In [1]:
%%time

from antifraud import read_adj_dict_from_file, read_into_list_of_tuples
from graph_algorithms import Graph

# set file names
batch  = '../paymo_input/batch_payment.txt'
stream = '../paymo_input/stream_payment.txt'

# Read files
# Turn on verboisity to see the lines that did not conform to the csv format (Note: 
# it's the same 5 or so lines over and over again)
batch0 = read_adj_dict_from_file (batch, verbose = False)
stream = read_into_list_of_tuples (stream, verbose = False) 

# attach
batch = Graph (batch0)

CPU times: user 33.5 s, sys: 1.58 s, total: 35.1 s
Wall time: 35.7 s


# Features

In [2]:
end = '\n'
msg_true = "verified" + end
msg_false = "unverified" + end
message = [msg_true,
           msg_false]

# feature 1
# msg_false = "unverified: You've never had a transaction with this user before. Are you sure you would like to proceed with this payment?"
# feature 2
# msg_false = 'unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?'
# feature 3
# msg_false = 'unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?'

## Feature 1
When anyone makes a payment to another user, they'll be notified if they've never made a transaction with that user before.

"unverified: You've never had a transaction with this user before. Are you sure you would like to proceed with this payment?"

In [ ]:
%%time

feature = batch.copy ()

output_file = 'output1.txt'

with open (output_file, 'w') as output:
    for pair in stream:
        lte = feature.if_lte_deg1 (pair)
        if not lte:
            feature.add_edge (pair)
        output.write (message [0] if lte else message [1])
print ('number of new nodes: {}\n'.format (feature.num_nodes - batch.num_nodes))

In [7]:
feature.num_nodes, batch.num_nodes

78032

## Feature 2

The PayMo team is concerned that these warnings could be annoying because there are many users who haven't had transactions, but are still in similar social networks. To account for this, PayMo would like you to also implement this feature. When users make a payment, they'll be notified when the other user is outside of their "2nd-degree network".

"unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?"

In [ ]:
%%time

output_file = 'output2.txt'
n = 2

feature = batch.copy ()

with open (output_file, 'w') as output:
    for edge in stream:
        output.write (msg_if_ltn (feature, edge, n, message, add_edge = True))
print ('number of new nodes: {}\n'.format (feature.num_nodes - batch.num_nodes))

## Feature 3

More generally, PayMo would like to extend this feature to larger social networks. Implement a feature to warn users only when they're outside the "4th degree friends network."

In [ ]:
%%time
msg_true = "verified"
output_file = 'output3.txt'
n = 4

start_length = feature.length
with open (output_file, 'w') as output:
    for edge in stream:
        for edge in stream:
        if feature.distance_lt_n (*edge, n):
            output.write (msg_true)
        else:
            output.write (msg_false)
            feature.add_edge (edge)
            
print ('number of new nodes: {}\n'.format (feature.length - start_length))